In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
data['rsi_14'] = data['rsi_14'].shift(1)

from talib import MA, SMA, EMA, WMA
data['ma_9'] = MA(data['close'], timeperiod=9)
data['ma_9'] = data['ma_9'].shift(1)
data['sma_9'] = SMA(data['close'], timeperiod=9)
data['sma_9'] = data['sma_9'].shift(1)
data['wma_9'] = WMA(data['close'], timeperiod=9)
data['wma_9'] = data['wma_9'].shift(1)

from talib import MACD
data['macd'], data['signal'], data['hist'] = MACD(data['close'])
data['macd'] = data['macd'].shift(1)
data['signal'] = data['signal'].shift(1)
data['hist'] = data['hist'].shift(1)

from talib import ADX
data['adx'] = ADX(data['high'], data['low'], data['close'])
data['adx'] = data['adx'].shift(1)

from talib import ATR
data['atr'] = ATR(high=data['high'], low=data['low'], close=data['close'], timeperiod=14)
data['atr'] = data['atr'].shift(1)

from talib import SAR
data['sar'] = SAR(high=data['high'], low=data['low'], acceleration=0.02, maximum=0.2)
data['sar'] = data['sar'].shift(1)

from talib import TEMA
data['tema'] = TEMA(data['close'], timeperiod=14)
data['tema'] = data['tema'].shift(1)

from talib import ROC
data['roc'] = ROC(data['close'], timeperiod=14)
data['roc'] = data['roc'].shift(1)

data.dropna(axis=0, inplace=True)

In [2]:
best_features_h = ('high', 'ma_9', 'sma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc')
best_features_l = ('low', 'ma_9', 'sma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc')

In [3]:
data_input_h = data[list(best_features_h)]
data_target_h = data[['high']]

data_input_l = data[list(best_features_l)]
data_target_l = data[['low']]

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler_h = MinMaxScaler()
data_scaled_h = scaler_h.fit_transform(data_input_h)

scaler_target_h = MinMaxScaler()
target_scaled_h = scaler_target_h.fit_transform(data_target_h)

scaler_l = MinMaxScaler()
data_scaled_l = scaler_l.fit_transform(data_input_l)

scaler_target_l = MinMaxScaler()
target_scaled_l = scaler_target_l.fit_transform(data_target_l)

In [7]:
seq_length = 4  # Number of time steps in each sequence

num_features_h = data_input_h.shape[1]
num_features_l = data_input_l.shape[1]

X_features_h = data_scaled_h
y_target_h = target_scaled_h
X_features_l = data_scaled_l
y_target_l = target_scaled_l

In [8]:
import numpy as np

# Create input sequences and targets
def create_sequences(features, target, seq_length):
    X_seq = []
    y_seq = []
    for i in range(len(features) - seq_length):
        X_seq.append(features[i:i+seq_length])  # Input sequence
        y_seq.append(target[i+seq_length]) # Target value (next data point)
    return np.array(X_seq), np.array(y_seq)

In [9]:
X_seq_h, y_seq_h = create_sequences(X_features_h, y_target_h, seq_length)
X_seq_l, y_seq_l = create_sequences(X_features_l, y_target_l, seq_length)

# Reshape X_seq to fit LSTM model input shape
X_seq_h = X_seq_h.reshape(X_seq_h.shape[0], seq_length, num_features_h)
X_seq_l = X_seq_l.reshape(X_seq_l.shape[0], seq_length, num_features_l)

In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
model_h = Sequential([
    LSTM(25, input_shape=(seq_length, num_features_h)),
    Dense(1)
])

# Compile the model
model_h.compile(optimizer='adam', loss='mean_squared_error')

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model with early stopping
model_h.fit(X_seq_h, y_seq_h, epochs=100, batch_size=45, validation_split=0.2, callbacks=[early_stopping])

2024-03-29 11:20:46.965677: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 11:20:47.043577: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 11:20:47.043659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 11:20:47.047145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 11:20:47.063249: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 11:20:48.663125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Epoch 1/100
289/289 [==============================] - 4s 7ms/step - loss: 0.0073 - val_loss: 8.8789e-05
Epoch 2/100
289/289 [==============================] - 1s 4ms/step - loss: 2.1234e-04 - val_loss: 6.7505e-05
Epoch 3/100
289/289 [==============================] - 1s 4ms/step - loss: 1.7494e-04 - val_loss: 5.0072e-05
Epoch 4/100
289/289 [==============================] - 1s 5ms/step - loss: 1.5957e-04 - val_loss: 4.7433e-05
Epoch 5/100
289/289 [==============================] - 1s 5ms/step - loss: 1.4992e-04 - val_loss: 9.7659e-05
Epoch 6/100
289/289 [==============================] - 1s 4ms/step - loss: 1.3958e-04 - val_loss: 3.5190e-05
Epoch 7/100
289/289 [==============================] - 1s 4ms/step - loss: 1.2867e-04 - val_loss: 4.4623e-05
Epoch 8/100
289/289 [==============================] - 1s 4ms/step - loss: 1.2073e-04 - val_loss: 2.6592e-05
Epoch 9/100
289/289 [==============================] - 1s 4ms/step - loss: 1.1176e-04 - val_loss: 2.2429e-05
Epoch 10/100
289/289 [=

In [11]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
model_l = Sequential([
    LSTM(25, input_shape=(seq_length, num_features_l)),
    Dense(1)
])

# Compile the model
model_l.compile(optimizer='adam', loss='mean_squared_error')

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model with early stopping
model_l.fit(X_seq_l, y_seq_l, epochs=100, batch_size=45, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
289/289 [==============================] - 4s 7ms/step - loss: 0.0092 - val_loss: 1.6820e-04
Epoch 2/100
289/289 [==============================] - 1s 5ms/step - loss: 2.2333e-04 - val_loss: 4.6294e-05
Epoch 3/100
289/289 [==============================] - 2s 6ms/step - loss: 2.0540e-04 - val_loss: 4.4793e-05
Epoch 4/100
289/289 [==============================] - 2s 5ms/step - loss: 1.9382e-04 - val_loss: 4.0514e-05
Epoch 5/100
289/289 [==============================] - 2s 7ms/step - loss: 1.7964e-04 - val_loss: 3.4264e-05
Epoch 6/100
289/289 [==============================] - 2s 6ms/step - loss: 1.6405e-04 - val_loss: 2.7477e-05
Epoch 7/100
289/289 [==============================] - 2s 6ms/step - loss: 1.5320e-04 - val_loss: 2.9457e-05
Epoch 8/100
289/289 [==============================] - 2s 6ms/step - loss: 1.4384e-04 - val_loss: 2.0183e-05
Epoch 9/100
289/289 [==============================] - 2s 6ms/step - loss: 1.3676e-04 - val_loss: 2.4559e-05
Epoch 10/100
289/289 [=

In [12]:
from talib import RSI
data_backup['rsi_14'] = RSI(data_backup['close'], timeperiod=14)
data_backup['rsi_14'] = data_backup['rsi_14'].shift(1)

from talib import MA, SMA, EMA, WMA
data_backup['ma_9'] = MA(data_backup['close'], timeperiod=9)
data_backup['ma_9'] = data_backup['ma_9'].shift(1)
data_backup['sma_9'] = SMA(data_backup['close'], timeperiod=9)
data_backup['sma_9'] = data_backup['sma_9'].shift(1)
data_backup['wma_9'] = WMA(data_backup['close'], timeperiod=9)
data_backup['wma_9'] = data_backup['wma_9'].shift(1)

from talib import MACD
data_backup['macd'], data_backup['signal'], data_backup['hist'] = MACD(data_backup['close'])
data_backup['macd'] = data_backup['macd'].shift(1)
data_backup['signal'] = data_backup['signal'].shift(1)
data_backup['hist'] = data_backup['hist'].shift(1)

from talib import ADX
data_backup['adx'] = ADX(data_backup['high'], data_backup['low'], data_backup['close'])
data_backup['adx'] = data_backup['adx'].shift(1)

from talib import ATR
data_backup['atr'] = ATR(high=data_backup['high'], low=data_backup['low'], close=data_backup['close'], timeperiod=14)
data_backup['atr'] = data_backup['atr'].shift(1)

from talib import SAR
data_backup['sar'] = SAR(high=data_backup['high'], low=data_backup['low'], acceleration=0.02, maximum=0.2)
data_backup['sar'] = data_backup['sar'].shift(1)

from talib import TEMA
data_backup['tema'] = TEMA(data_backup['close'], timeperiod=14)
data_backup['tema'] = data_backup['tema'].shift(1)

from talib import ROC
data_backup['roc'] = ROC(data_backup['close'], timeperiod=14)
data_backup['roc'] = data_backup['roc'].shift(1)

data_backup.dropna(axis=0, inplace=True)

In [13]:
data_backup_h_scaled = scaler_h.transform(data_backup[list(best_features_h)])
target_backup_h_scaled = scaler_target_h.transform(data_backup[['high']])
X_b_h, y_b_h = create_sequences(data_backup_h_scaled, target_backup_h_scaled, seq_length)
X_b_h = X_b_h.reshape(X_b_h.shape[0], seq_length, num_features_h)

data_backup_l_scaled = scaler_l.transform(data_backup[list(best_features_l)])
target_backup_l_scaled = scaler_target_l.transform(data_backup[['low']])
X_b_l, y_b_l = create_sequences(data_backup_l_scaled, target_backup_l_scaled, seq_length)
X_b_l = X_b_l.reshape(X_b_l.shape[0], seq_length, num_features_l)

In [14]:
y_pred_h = model_h.predict(X_b_h)
y_pred_h = scaler_target_h.inverse_transform(y_pred_h)

y_pred_l = model_l.predict(X_b_l)
y_pred_l = scaler_target_l.inverse_transform(y_pred_l)

252/252 [==============================] - 1s 2ms/step


In [24]:
data_backup.reset_index(inplace=True)

In [16]:
# graphics?

In [15]:
from sklearn.metrics import mean_squared_error, accuracy_score

score_h = mean_squared_error(data_backup[['high']].iloc[seq_length:], y_pred_h)
score_l = mean_squared_error(data_backup[['low']].iloc[seq_length:], y_pred_l)

print(f'score high: {score_h}')
print(f'score low: {score_l}')

score high: 153.24242150106053
score low: 124.35153152658478


In [27]:
y_pred_h_series = pd.Series(y_pred_h.reshape(1,-1)[0])
y_pred_l_series = pd.Series(y_pred_l.reshape(1,-1)[0])
data_backup['pred_highs'] = y_pred_h_series
data_backup['pred_lows'] = y_pred_l_series
data_backup.to_excel('lstm-multiple-factor-higs&lows.xlsx')